In [1]:
import numpy as np
import pandas as pd
from pandas.io import sql
import sqlalchemy as sa
import yaml as yml
import json
import demjson
get_ipython().magic(u'matplotlib inline')
import scipy.stats as ssp
from matplotlib import pyplot as plt
from __future__ import division

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("./bfas_IIFA6.csv")

In [4]:
df.head()

,Label,Item,N,A,C,E,O,OA
0,BFAS.95.,"""I see beauty in things that others might not...",-0.048092,0.174424,0.013136,0.136941,0.258037,0.485217
1,BFAS.96.,"""I need a creative outlet.""",-0.201576,0.249581,-0.027287,0.144686,0.277920,0.433204
2,BFAS.32.,I hate to seem pushy.,0.198721,0.180869,-0.023733,-0.386263,-0.040147,0.398678
3,BFAS.94.,"""I get deeply immersed in music.""",-0.192445,0.078845,0.013724,0.096318,0.184047,0.390981
4,BFAS.33.,I avoid imposing my will on others.,0.250159,0.090589,0.001326,-0.483260,0.009156,0.348330


In [5]:
#df.columns = ['label','item','N','EA','A','C','OA','OI','EE']

In [6]:
# df[['N','EA','A','C','OA','OI','EE']].iloc[0].idxmax()

In [7]:
df['absN'] = abs(df['N'])
df['absA'] = abs(df['A'])
df['absC'] = abs(df['C'])
df['absE'] = abs(df['E'])
df['absO'] = abs(df['O'])
df['absOA'] = abs(df['OA'])

In [8]:
df['max_factor'] = df[['absN','absA','absC','absE','absO','absOA']].idxmax(axis=1)

In [9]:
df.tail()

,Label,Item,N,A,C,E,O,OA,absN,absA,absC,absE,absO,absOA,max_factor
95,BFAS.13.,"I worry about things.,",0.581253,-0.041815,-0.069676,0.203197,0.125249,-0.208785,0.581253,0.041815,0.069676,0.203197,0.125249,0.208785,absN
96,BFAS.80.,"""I hold back my opinions.""",0.009256,0.124194,0.101350,0.389443,0.223401,-0.209126,0.009256,0.124194,0.101350,0.389443,0.223401,0.209126,absE
97,BFAS.77.,"""I do not have an assertive personality.""",-0.101473,0.058952,0.138994,0.550928,0.148573,-0.210548,0.101473,0.058952,0.138994,0.550928,0.148573,0.210548,absE
98,BFAS.16.,"I am afraid of many things.,",0.507539,0.001890,-0.059370,0.167534,0.237961,-0.269156,0.507539,0.001890,0.059370,0.167534,0.237961,0.269156,absN
99,BFAS.11.,"I am filled with doubts about things.,",0.483076,0.082949,0.204136,0.212518,0.105989,-0.293968,0.483076,0.082949,0.204136,0.212518,0.105989,0.293968,absN


In [10]:
df = df.drop(['absN','absA','absC','absE','absO','absOA'],axis=1)

In [11]:
df.head()

,Label,Item,N,A,C,E,O,OA,max_factor
0,BFAS.95.,"""I see beauty in things that others might not...",-0.048092,0.174424,0.013136,0.136941,0.258037,0.485217,absOA
1,BFAS.96.,"""I need a creative outlet.""",-0.201576,0.249581,-0.027287,0.144686,0.277920,0.433204,absOA
2,BFAS.32.,I hate to seem pushy.,0.198721,0.180869,-0.023733,-0.386263,-0.040147,0.398678,absOA
3,BFAS.94.,"""I get deeply immersed in music.""",-0.192445,0.078845,0.013724,0.096318,0.184047,0.390981,absOA
4,BFAS.33.,I avoid imposing my will on others.,0.250159,0.090589,0.001326,-0.483260,0.009156,0.348330,absE


In [12]:
df['maxfactor'] = df['max_factor'].apply(lambda x: x[3:])

In [13]:
df = df.drop(['max_factor'],axis=1)

In [14]:
df.head()

,Label,Item,N,A,C,E,O,OA,maxfactor
0,BFAS.95.,"""I see beauty in things that others might not...",-0.048092,0.174424,0.013136,0.136941,0.258037,0.485217,OA
1,BFAS.96.,"""I need a creative outlet.""",-0.201576,0.249581,-0.027287,0.144686,0.277920,0.433204,OA
2,BFAS.32.,I hate to seem pushy.,0.198721,0.180869,-0.023733,-0.386263,-0.040147,0.398678,OA
3,BFAS.94.,"""I get deeply immersed in music.""",-0.192445,0.078845,0.013724,0.096318,0.184047,0.390981,OA
4,BFAS.33.,I avoid imposing my will on others.,0.250159,0.090589,0.001326,-0.483260,0.009156,0.348330,E


In [15]:
col_dict={"N":2,"A":3,"C":4,"E":5,"O":6,"OA":7}

In [16]:
def get_max_loading(x):
    mfactor = x[8]
    col = col_dict[str(mfactor)]
    return x[col]

In [17]:

df['max_factor_loading'] = df.apply(lambda x:  get_max_loading(x),axis=1)

In [18]:
df.head()

,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
0,BFAS.95.,"""I see beauty in things that others might not...",-0.048092,0.174424,0.013136,0.136941,0.258037,0.485217,OA,0.485217
1,BFAS.96.,"""I need a creative outlet.""",-0.201576,0.249581,-0.027287,0.144686,0.277920,0.433204,OA,0.433204
2,BFAS.32.,I hate to seem pushy.,0.198721,0.180869,-0.023733,-0.386263,-0.040147,0.398678,OA,0.398678
3,BFAS.94.,"""I get deeply immersed in music.""",-0.192445,0.078845,0.013724,0.096318,0.184047,0.390981,OA,0.390981
4,BFAS.33.,I avoid imposing my will on others.,0.250159,0.090589,0.001326,-0.483260,0.009156,0.348330,E,-0.483260


In [19]:
Neuroticism = df[df['maxfactor']=='N'].sort('max_factor_loading',ascending=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [20]:
Neuroticism

,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
62,BFAS.2.,"I get upset easily.,",0.838041,-0.005847,-0.064846,-0.012647,0.090872,-0.035117,N,0.838041
52,BFAS.6.,"I can be stirred up easily.,",0.771733,0.002709,-0.039947,-0.085114,0.074426,0.014850,N,0.771733
40,BFAS.F7.,"I rarely get irritated.,",0.770614,0.060822,-0.121574,0.007510,-0.141380,0.080718,N,0.770614
30,BFAS.1.,"I get angry easily.,",0.768107,0.178615,-0.071662,-0.132700,0.006589,0.133519,N,0.768107
54,BFAS.5.,"I get agitated easily.,",0.766725,0.128458,-0.030452,-0.035741,0.072863,0.011549,N,0.766725
10,BFAS.9.,"I rarely lose my composure.,",0.759216,-0.108484,0.086314,-0.043548,-0.042932,0.272097,N,0.759216
92,BFAS.4.,"I am a person whose moods go up and down easily.,",0.736918,0.081273,0.091497,-0.088008,0.062837,-0.185233,N,0.736918
88,BFAS.3.,"I change my mood a lot.,",0.735144,0.081538,0.128222,-0.119275,0.045762,-0.152014,N,0.735144
24,BFAS.8.,"I keep my emotions under control,",0.710027,-0.216870,0.122963,-0.063558,0.046482,0.176914,N,0.710027
32,BFAS.10.,"I am not easily annoyed.,",0.699495,0.121019,-0.064699,0.052367,-0.057235,0.112206,N,0.699495


In [23]:
Agree = df[df['maxfactor']=='A'].sort('max_factor_loading',ascending=False)
Agree

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
76,BFAS.26.,"I am not interested in other people's problems.,",-0.072769,0.864884,-0.016558,-0.024375,0.052538,-0.096215,A,0.864884
75,BFAS.27.,I can't be bothered with other's needs.,-0.027559,0.841080,-0.034026,-0.098620,0.063135,-0.092022,A,0.841080
71,BFAS.28.,I am indifferent to the feelings of others.,-0.003490,0.812836,0.043254,-0.167661,0.195879,-0.056802,A,0.812836
21,BFAS.23.,"I sympathize with others' feelings.,",0.011250,0.771438,0.078298,-0.012822,-0.041773,0.195540,A,0.771438
45,BFAS.24.,"I take an interest in other people's lives.,",-0.039420,0.757979,0.020287,0.169342,-0.026138,0.051497,A,0.757979
53,BFAS.25.,"I like to do things for others.,",0.090856,0.731183,0.039737,0.045526,-0.032079,0.013879,A,0.731183
41,BFAS.22.,"I inquire about others' well-being.,",0.087907,0.699948,0.073923,0.146604,-0.072949,0.076077,A,0.699948
42,BFAS.29.,I take no time for others.,0.054834,0.648139,0.035328,0.051876,-0.000444,0.070002,A,0.648139
9,BFAS.21.,"I feel others' emotions.,",-0.061310,0.639097,0.064127,0.090431,-0.016015,0.287292,A,0.639097
70,BFAS.40.,I am out for my own personal gain.,0.108380,0.592703,-0.061207,-0.264646,0.031711,-0.053900,A,0.592703


In [24]:
Conci = df[df['maxfactor']=='C'].sort('max_factor_loading',ascending=False)
Conci

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
58,BFAS.52.,I keep things tidy.,-0.017631,-0.019562,0.762788,0.003228,0.011324,-0.007458,C,0.762788
63,BFAS.51.,I like order.,-0.072377,0.058290,0.740945,-0.155583,-0.112443,-0.037869,C,0.740945
38,BFAS.53.,I follow a schedule.,-0.062938,-0.004097,0.689559,0.094505,-0.116846,0.086818,C,0.689559
31,BFAS.56.,I want every detail taken care of.,-0.166836,-0.048370,0.685885,0.051953,0.019386,0.128766,C,0.685885
90,BFAS.59.,I am not bothered by disorder.,-0.252914,0.109308,0.670557,-0.123427,-0.039203,-0.165498,C,0.670557
22,BFAS.54.,I want everything to be 'just right'.,-0.243076,-0.064281,0.627527,0.103023,-0.175728,0.179458,C,0.627527
83,BFAS.58.,I am not bothered by messy people.,-0.253649,0.004581,0.622801,-0.103768,0.051743,-0.116818,C,0.622801
46,BFAS.42.,I finish what I start.,0.203141,0.016081,0.622738,0.114608,0.128005,0.044140,C,0.622738
84,BFAS.57.,I leave my belongings around.,0.025514,0.034246,0.622308,-0.049176,0.077287,-0.130749,C,0.622308
18,BFAS.55.,I see that rules are observed.,0.041406,0.066428,0.610339,0.053124,-0.261482,0.217516,C,0.610339


In [22]:
Extra = df[df['maxfactor']=='E'].sort('max_factor_loading',ascending=False)
Extra

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
13,BFAS.73.,"""I know how to captivate people.""",0.023158,0.052648,-0.024920,0.717644,0.110591,0.251231,E,0.717644
82,BFAS.71.,"""I take charge.""",-0.016760,0.012699,0.203248,0.703703,0.076641,-0.108869,E,0.703703
57,BFAS.74.,"""I see myself as a good leader.""",0.032528,0.011003,0.144621,0.683805,0.143353,-0.005006,E,0.683805
49,BFAS.75.,"""I can talk others into doing things.""",-0.124957,0.002017,-0.023850,0.664168,0.106942,0.033446,E,0.664168
56,BFAS.76.,"""I am the first to act.""",0.012299,0.024558,0.180710,0.631268,0.052678,0.005556,E,0.631268
66,BFAS.78.,"""I lack the talent for influencing people.""",-0.000380,0.144032,-0.021552,0.627306,0.227882,-0.042721,E,0.627306
43,BFAS.72.,"""I have a strong personality.""",-0.100790,-0.023334,0.057983,0.551296,0.299239,0.063253,E,0.551296
97,BFAS.77.,"""I do not have an assertive personality.""",-0.101473,0.058952,0.138994,0.550928,0.148573,-0.210548,E,0.550928
61,BFAS.61.,I make friends easily.,0.222969,0.377024,-0.008241,0.534925,-0.119685,-0.026122,E,0.534925
25,BFAS.64.,"""I have a lot of fun.""",0.239554,0.165126,0.039781,0.530383,-0.047160,0.165931,E,0.530383


In [25]:
Open = df[df['maxfactor']=='O'].sort('max_factor_loading',ascending=False)
Open

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
64,BFAS.F87.,"""I have difficulty understanding abstract ide...",0.116759,0.108699,-0.030879,-0.101902,0.686999,-0.037977,O,0.686999
69,BFAS.89.,"""I avoid difficult reading material.""",-0.019139,0.096238,-0.039512,-0.006047,0.650285,-0.050432,O,0.650285
51,BFAS.F83.,"""I like to solve complex problems.""",0.023722,-0.022859,-0.037639,0.138847,0.629070,0.022817,O,0.629070
48,BFAS.81.,"""I am quick to understand things.""",0.150328,-0.077877,0.063332,0.231277,0.592971,0.040397,O,0.592971
77,BFAS.90.,"""I learn things slowly.""",0.185677,0.035571,0.096719,0.076392,0.579007,-0.103806,O,0.579007
34,BFAS.82.,"""I can handle a lot of information.""",0.164290,-0.033866,0.112874,0.182459,0.572439,0.109135,O,0.572439
35,BFAS.88.,"""I avoid philosophical discussions.""",0.076779,0.080548,-0.070111,0.069159,0.555605,0.104478,O,0.555605
16,BFAS.F84.,"""I have a rich vocabularly.""",-0.054602,0.095018,0.026498,0.139101,0.459616,0.225946,O,0.459616
33,BFAS.F85.,"""I think quickly.""",0.087463,-0.063252,0.182044,0.244302,0.452251,0.111313,O,0.452251
23,BFAS.F86.,"""I formulate ideas clearly.""",0.192090,-0.009266,0.300256,0.191333,0.382072,0.178818,O,0.382072


In [26]:
OpenA = df[df['maxfactor']=='OA'].sort('max_factor_loading',ascending=False)
OpenA

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,C,E,O,OA,maxfactor,max_factor_loading
0,BFAS.95.,"""I see beauty in things that others might not...",-0.048092,0.174424,0.013136,0.136941,0.258037,0.485217,OA,0.485217
1,BFAS.96.,"""I need a creative outlet.""",-0.201576,0.249581,-0.027287,0.144686,0.277920,0.433204,OA,0.433204
2,BFAS.32.,I hate to seem pushy.,0.198721,0.180869,-0.023733,-0.386263,-0.040147,0.398678,OA,0.398678
3,BFAS.94.,"""I get deeply immersed in music.""",-0.192445,0.078845,0.013724,0.096318,0.184047,0.390981,OA,0.390981
8,BFAS.91.,"""I enjoy the beauty of nature.""",0.009211,0.260889,0.103576,0.010723,0.263055,0.314258,OA,0.314258


In [ ]:
333

In [46]:
df2 = pd.read_csv("./bfas_IIFA5.csv")

In [47]:
#df2.columns = ['label','item','N','A','E','C','O','EE']

In [48]:
df2.head()

,Label,Item,N,A,E,C,O
0,BFAS.F87.,"""I have difficulty understanding abstract ide...",0.186014,0.020251,-0.057340,-0.048574,0.645523
1,BFAS.F83.,"""I like to solve complex problems.""",0.065980,-0.055141,0.158774,-0.047623,0.622522
2,BFAS.89.,"""I avoid difficult reading material.""",0.051072,0.011282,0.044368,-0.055955,0.614715
3,BFAS.81.,"""I am quick to understand things.""",0.184157,-0.083136,0.231797,0.055177,0.594301
4,BFAS.82.,"""I can handle a lot of information.""",0.178517,-0.016592,0.158892,0.108217,0.591361


In [49]:
df2['absN'] = abs(df2['N'])
df2['absA'] = abs(df2['A'])
df2['absE'] = abs(df2['E'])
df2['absC'] = abs(df2['C'])
df2['absO'] = abs(df2['O'])


df2['max_factor'] = df2[['absN','absA','absE','absC','absO']].idxmax(axis=1)

In [50]:
df2 = df2.drop(['absN','absA','absE','absC','absO'],axis=1)

In [51]:
df2['maxfactor'] = df2['max_factor'].apply(lambda x: x[3:])

In [52]:
df2 = df2.drop(['max_factor'],axis=1)

In [53]:
df2.head()

,Label,Item,N,A,E,C,O,maxfactor
0,BFAS.F87.,"""I have difficulty understanding abstract ide...",0.186014,0.020251,-0.057340,-0.048574,0.645523,O
1,BFAS.F83.,"""I like to solve complex problems.""",0.065980,-0.055141,0.158774,-0.047623,0.622522,O
2,BFAS.89.,"""I avoid difficult reading material.""",0.051072,0.011282,0.044368,-0.055955,0.614715,O
3,BFAS.81.,"""I am quick to understand things.""",0.184157,-0.083136,0.231797,0.055177,0.594301,O
4,BFAS.82.,"""I can handle a lot of information.""",0.178517,-0.016592,0.158892,0.108217,0.591361,O


In [54]:
col_dict2={"N":2,"A":3,"E":4,"C":5,"O":6}

In [55]:
def get_max_loading(x):
    mfactor = x[7]
    col = col_dict2[str(mfactor)]
    return x[col]
df2['max_factor_loading'] = df2.apply(lambda x:  get_max_loading(x),axis=1)

In [56]:
df2.head()

,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
0,BFAS.F87.,"""I have difficulty understanding abstract ide...",0.186014,0.020251,-0.057340,-0.048574,0.645523,O,0.645523
1,BFAS.F83.,"""I like to solve complex problems.""",0.065980,-0.055141,0.158774,-0.047623,0.622522,O,0.622522
2,BFAS.89.,"""I avoid difficult reading material.""",0.051072,0.011282,0.044368,-0.055955,0.614715,O,0.614715
3,BFAS.81.,"""I am quick to understand things.""",0.184157,-0.083136,0.231797,0.055177,0.594301,O,0.594301
4,BFAS.82.,"""I can handle a lot of information.""",0.178517,-0.016592,0.158892,0.108217,0.591361,O,0.591361


In [57]:
Neuroticism2= df2[df2['maxfactor']=='N'].sort('max_factor_loading',ascending=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [58]:
Neuroticism2

,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
48,BFAS.2.,"I get upset easily.,",0.851685,-0.002846,-0.049888,-0.071144,0.071297,N,0.851685
79,BFAS.4.,"I am a person whose moods go up and down easily.,",0.795897,0.009543,-0.068760,0.076662,-0.000210,N,0.795897
80,BFAS.3.,"I change my mood a lot.,",0.783329,0.022090,-0.111305,0.114659,-0.007882,N,0.783329
52,BFAS.5.,"I get agitated easily.,",0.770448,0.145799,-0.088292,-0.035523,0.066228,N,0.770448
51,BFAS.6.,"I can be stirred up easily.,",0.769296,0.017868,-0.133903,-0.043039,0.067995,N,0.769296
91,BFAS.F7.,"I rarely get irritated.,",0.731501,0.131608,-0.078545,-0.115958,-0.119070,N,0.731501
63,BFAS.1.,"I get angry easily.,",0.729820,0.238908,-0.226092,-0.070301,0.033946,N,0.729820
65,BFAS.9.,"I rarely lose my composure.,",0.665194,0.029124,-0.178227,0.099241,0.027232,N,0.665194
42,BFAS.15.,"I become overwhelmed by events.,",0.664168,-0.133260,0.170965,0.085291,0.105638,N,0.664168
83,BFAS.10.,"I am not easily annoyed.,",0.660606,0.198612,-0.037430,-0.058952,-0.029075,N,0.660606


In [59]:
Agree2 = df2[df2['maxfactor']=='A'].sort('max_factor_loading',ascending=False)
Agree2

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
74,BFAS.23.,"I sympathize with others' feelings.,",-0.024960,0.836980,-0.096643,0.079688,0.011574,A,0.836980
66,BFAS.26.,"I am not interested in other people's problems.,",-0.013399,0.786307,-0.001925,-0.031676,0.025530,A,0.786307
81,BFAS.24.,"I take an interest in other people's lives.,",-0.033461,0.784526,0.131300,0.015254,-0.008226,A,0.784526
62,BFAS.27.,I can't be bothered with other's needs.,0.030524,0.756402,-0.076728,-0.048549,0.033980,A,0.756402
53,BFAS.21.,"I feel others' emotions.,",-0.125782,0.754336,-0.018698,0.071989,0.065407,A,0.754336
84,BFAS.22.,"I inquire about others' well-being.,",0.081259,0.746509,0.091500,0.071373,-0.050054,A,0.746509
82,BFAS.25.,"I like to do things for others.,",0.106665,0.730751,0.017000,0.032136,-0.028908,A,0.730751
32,BFAS.28.,I am indifferent to the feelings of others.,0.055274,0.725063,-0.145823,0.026138,0.164622,A,0.725063
71,BFAS.29.,I take no time for others.,0.054177,0.670958,0.010376,0.030824,0.017534,A,0.670958
30,BFAS.30.,I don't have a soft side.,-0.074312,0.629478,-0.091414,0.035943,0.165586,A,0.629478


In [60]:
Extra2 = df2[df2['maxfactor']=='E'].sort('max_factor_loading',ascending=False)
Extra2 

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
56,BFAS.71.,"""I take charge.""",0.023831,0.042759,0.722190,0.200693,0.061591,E,0.722190
33,BFAS.74.,"""I see myself as a good leader.""",0.046711,0.077962,0.666929,0.145602,0.156580,E,0.666929
40,BFAS.75.,"""I can talk others into doing things.""",-0.125615,0.082012,0.642108,-0.019726,0.132587,E,0.642108
23,BFAS.78.,"""I lack the talent for influencing people.""",0.034551,0.180060,0.627333,-0.025582,0.224280,E,0.627333
44,BFAS.77.,"""I do not have an assertive personality.""",-0.023881,0.020983,0.615344,0.127615,0.098488,E,0.615344
50,BFAS.76.,"""I am the first to act.""",0.016704,0.097274,0.608892,0.184731,0.068536,E,0.608892
25,BFAS.73.,"""I know how to captivate people.""",-0.037063,0.232063,0.605379,-0.007358,0.193448,E,0.605379
20,BFAS.72.,"""I have a strong personality.""",-0.094467,0.039826,0.533715,0.059853,0.322697,E,0.533715
22,BFAS.79.,"""I wait for others to lead the way.""",0.205475,-0.041086,0.528709,0.080341,0.273813,E,0.528709
89,BFAS.61.,I make friends easily.,0.231136,0.445700,0.495322,-0.009405,-0.112876,E,0.495322


In [61]:
Conci2 = df2[df2['maxfactor']=='C'].sort('max_factor_loading',ascending=False)
Conci2

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
77,BFAS.52.,I keep things tidy.,-0.007872,-0.025209,0.004301,0.762046,0.004990,C,0.762046
92,BFAS.51.,I like order.,-0.061711,0.025039,-0.142660,0.737419,-0.122965,C,0.737419
85,BFAS.53.,I follow a schedule.,-0.090979,0.046638,0.057213,0.698048,-0.086872,C,0.698048
58,BFAS.56.,I want every detail taken care of.,-0.195939,-0.001551,0.017593,0.691793,0.057004,C,0.691793
86,BFAS.59.,I am not bothered by disorder.,-0.195302,0.015397,-0.053739,0.656083,-0.087588,C,0.656083
90,BFAS.54.,I want everything to be 'just right'.,-0.303916,0.025740,0.045533,0.639062,-0.114594,C,0.639062
39,BFAS.42.,I finish what I start.,0.206577,0.037908,0.086354,0.625014,0.135074,C,0.625014
94,BFAS.55.,I see that rules are observed.,-0.034538,0.181687,-0.040162,0.620123,-0.188329,C,0.620123
64,BFAS.57.,I leave my belongings around.,0.076147,-0.033981,-0.005148,0.612650,0.033119,C,0.612650
72,BFAS.58.,I am not bothered by messy people.,-0.207796,-0.070092,-0.042646,0.610323,0.013700,C,0.610323


In [62]:
Open2 = df2[df2['maxfactor']=='O'].sort('max_factor_loading',ascending=False)
Open2

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Label,Item,N,A,E,C,O,maxfactor,max_factor_loading
0,BFAS.F87.,"""I have difficulty understanding abstract ide...",0.186014,0.020251,-0.057340,-0.048574,0.645523,O,0.645523
1,BFAS.F83.,"""I like to solve complex problems.""",0.065980,-0.055141,0.158774,-0.047623,0.622522,O,0.622522
2,BFAS.89.,"""I avoid difficult reading material.""",0.051072,0.011282,0.044368,-0.055955,0.614715,O,0.614715
3,BFAS.81.,"""I am quick to understand things.""",0.184157,-0.083136,0.231797,0.055177,0.594301,O,0.594301
4,BFAS.82.,"""I can handle a lot of information.""",0.178517,-0.016592,0.158892,0.108217,0.591361,O,0.591361
5,BFAS.88.,"""I avoid philosophical discussions.""",0.092892,0.082040,0.054375,-0.075974,0.568215,O,0.568215
6,BFAS.90.,"""I learn things slowly.""",0.263349,-0.045571,0.125366,0.078924,0.532415,O,0.532415
7,BFAS.F84.,"""I have a rich vocabularly.""",-0.079507,0.157826,0.082877,0.028818,0.513697,O,0.513697
8,BFAS.F85.,"""I think quickly.""",0.090557,-0.027617,0.217129,0.180362,0.478689,O,0.478689
9,BFAS.92.,"""I believe in the importance of art.""",-0.175237,0.407727,-0.044299,-0.037402,0.448022,O,0.448022


In [64]:
Neuroticism2['Label'].values.tolist()

['BFAS.2.',
 'BFAS.4.',
 'BFAS.3.',
 'BFAS.5.',
 'BFAS.6.',
 'BFAS.F7.',
 'BFAS.1.',
 'BFAS.9.',
 'BFAS.15.',
 'BFAS.10.',
 'BFAS.12.',
 'BFAS.8.',
 'BFAS.13.',
 'BFAS.16.',
 'BFAS.17.',
 'BFAS.11.',
 'BFAS.19.',
 'BFAS.18.',
 'BFAS.14.',
 'BFAS.20.']

In [65]:
Agree2['Label'].values.tolist()

['BFAS.23.',
 'BFAS.26.',
 'BFAS.24.',
 'BFAS.27.',
 'BFAS.21.',
 'BFAS.22.',
 'BFAS.25.',
 'BFAS.28.',
 'BFAS.29.',
 'BFAS.30.',
 'BFAS.67.',
 'BFAS.63.',
 'BFAS.62.',
 'BFAS.40.',
 'BFAS.66.',
 'BFAS.100.',
 'BFAS.68.',
 'BFAS.70.',
 'BFAS.96.',
 'BFAS.65.',
 'BFAS.97.',
 'BFAS.69.',
 'BFAS.91.']

In [66]:
Extra2['Label'].values.tolist()

['BFAS.71.',
 'BFAS.74.',
 'BFAS.75.',
 'BFAS.78.',
 'BFAS.77.',
 'BFAS.76.',
 'BFAS.73.',
 'BFAS.72.',
 'BFAS.79.',
 'BFAS.61.',
 'BFAS.80.',
 'BFAS.64.',
 'BFAS.37.',
 'BFAS.35.',
 'BFAS.39.',
 'BFAS.38.',
 'BFAS.36.',
 'BFAS.32.',
 'BFAS.34.',
 'BFAS.33.']

In [67]:
Conci2['Label'].values.tolist()

['BFAS.52.',
 'BFAS.51.',
 'BFAS.53.',
 'BFAS.56.',
 'BFAS.59.',
 'BFAS.54.',
 'BFAS.42.',
 'BFAS.55.',
 'BFAS.57.',
 'BFAS.58.',
 'BFAS.41.',
 'BFAS.45.',
 'BFAS.46.',
 'BFAS.60.',
 'BFAS.47.',
 'BFAS.43.',
 'BFAS.49.',
 'BFAS.50.',
 'BFAS.31.',
 'BFAS.44.',
 'BFAS.48.']

In [68]:
Open2['Label'].values.tolist()

['BFAS.F87.',
 'BFAS.F83.',
 'BFAS.89.',
 'BFAS.81.',
 'BFAS.82.',
 'BFAS.88.',
 'BFAS.90.',
 'BFAS.F84.',
 'BFAS.F85.',
 'BFAS.92.',
 'BFAS.93.',
 'BFAS.F86.',
 'BFAS.98.',
 'BFAS.95.',
 'BFAS.99.',
 'BFAS.94.']